In [74]:

import mysql.connector
from mysql.connector import errorcode
import logging


logging.basicConfig(level=logging.DEBUG, filename='effigySync.log', format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)


In [75]:
import os
SOURCEPATH='Y:/server/EFFIGY_PICS'
#SOURCEPATH = 'C:/Users/user/Pictures'


In [76]:
def fixSlashes(val):
    val = val.replace('\\', '/')
    return val    

In [77]:


folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(SOURCEPATH):
        print(r)


Y:/server/EFFIGY_PICS
Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006
Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\20101105_CUBA_ZDJECIA
Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\20101105_CUBA_ZDJECIA\100OLYMP
Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\CUBA 2012
Y:/server/EFFIGY_PICS\Zdjecia
Y:/server/EFFIGY_PICS\Zdjecia\Minolta
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20030501_FIDELITY_STATEK
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20030601_FIDELITY_ZDJECIA _SIATKA_WONDERLAND
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\2004 Ready First pics
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20040101 Impreza Hinduska
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20040701_FIDELITY_WANDERLAND_ERIC
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20041120 Eryk spiewa latawce wino
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20050403 Swieta Sylwester Urodziny
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20050512 Dom Jola CNtower rodzinne
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20050522 Jola Niagara
Y:/server/EFFIGY_PICS\Zdjecia\Minolta\20050627 Wonderland
Y

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\connector
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\context
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\defaultcontext
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\host
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\images
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\logger
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\realm
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\resources
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\server
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\service
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps\admin\users
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server\webapps

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-5\resources\confinstall
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-5\resources\deployer
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-5\resources\mbeans
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\etc
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\src
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\src\bin
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\src\conf
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\src\share
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\catalina\src\share\org
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\server
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\service
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\users
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\valve
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\WEB-INF
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\WEB-INF\classes
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\WEB-INF\classes\org
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\WEB-INF\classes\org\apache
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admin\WEB-INF\classes\org\apache\webapp
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-catalina\webapps\admi

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\ajp
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\ajp\tomcat33
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\ajp\tomcat4
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\ajp\tomcat4\config
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\jk
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\jk\apr
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache\jk\common
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\jk\java\org\apache

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\buf
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\buf\res
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\collections
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\compat
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\http
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\http\mapper
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util\java\org\apache\tomcat\util\http\res
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-tomcat-connectors\util

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\sessions
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\simpletag
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\snp
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\tagplugin
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\cal
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\checkbox
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\colors
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\compressionFilters
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\dates
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\jsp-examples\WEB-INF\classes\error
Y:/s

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\ajp
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\ajp\tomcat33
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\ajp\tomcat4
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\ajp\tomcat4\config
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\catalina
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\catalina\ant
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\docs\api\org\apache\catalina

Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jspapi\javax\servlet\jsp\tagext\class-use
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jspapi\javax\servlet\jsp\tagext\doc-files
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jspapi\resources
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\printer
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax\servlet
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax\servlet\class-use
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax\servlet\http
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax\servlet\http\class-use
Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\resourc

Y:/server/EFFIGY_PICS\Zdjecia2008_II\20090801 African lion Safari
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20090807 Muskoka Cottage
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20090912_Air_Show_Urodziny_Krzysiek
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20090914 Urodzainy Shelly
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20091123Top ONE Koncert
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20091130 Indyk Childrens party
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20091228 Quebec MontTremblant
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100628 Adam Pilka
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100628 Do Polski
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100628 Do Polski\enlarge
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100703 Polska_Sylwia_Dziadek
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100703 Polska_Sylwia_Dziadek\CONVERTED
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100715_NowaCamera_NiagaraRoma
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100720 AdamPilka
Y:/server/EFFIGY_PICS\Zdjecia2008_II\20100725 Ogrodek
Y:/server/EFFIGY_PICS\Zdjecia2008_II\2

In [78]:
def getID(tbl,thisid, key,val):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [79]:
getID("loc_folder","folder_id", "folder_path","C:/Users/user/Pictures/2017-11111111111")

[]


-1

In [80]:
def insertRoot (root):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_root values (null, %s)"
        root = fixSlashes(root)
        #print (root)
        mycursor.execute(ssql, (root,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_root","root_id", "root",root)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Root-Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Root - DB Closed")
        return (newid)

In [81]:
def insertFolder (root_id,root,folder,folderPath):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_folder values (null,%s, %s,%s,%s)"
        root = fixSlashes(root)
        folder = fixSlashes(folder)
        folderPath = fixSlashes(folderPath)
        #print (ssql)
        mycursor.execute(ssql, (root_id,root,folder,folderPath,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_folder","folder_id", "folder_path",folderPath)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        
        return (newid)

In [82]:
getID("loc_folder","folder_id", "folder_path","C:ggg/Uffsssehhffrsggs/user/Pictures/2017-11111111111")

[]


-1

In [86]:
def skipRoot(root):
    skip=False
    toSkip="Y:\server\EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA"
    if len(root) >= len(toSkip):
        if toSkip == root[:len(toSkip)]:
            skip=True
    return(skip)

In [83]:
#newid = insertFolder(1, "C:/Users/user/Pictures","2017-1111", "C:ggg/Uffsssehhffrsggs/user/Pictures/2017-1221111111111")
#newid

In [84]:
def importFolders():
    for r, d, f in os.walk(SOURCEPATH):
        #print("dir",d,r)
        if len(d) >0:
            if  skipRoot(r):
                print("Skipped Root:",r)
            else:
                idR = insertRoot(r)
                print("Processed Root:",idR,r)
            for dir in d:
                #print ("will INSERT dir:",idR,dir)
                print("about to insert", idR,r,dir,r + '/' + dir)
                idF = insertFolder(idR,r,dir,r + "/"+ dir)
                print("Created folder",idF)
                
        
    

In [85]:
importFolders()

[(7,)]
this is root 7 Y:/server/EFFIGY_PICS
about to insert 7 Y:/server/EFFIGY_PICS FILMY_MAMY_20121006 Y:/server/EFFIGY_PICS/FILMY_MAMY_20121006
[(69,)]
Created folder 69
about to insert 7 Y:/server/EFFIGY_PICS Zdjecia Y:/server/EFFIGY_PICS/Zdjecia
[(70,)]
Created folder 70
about to insert 7 Y:/server/EFFIGY_PICS Zdjecia2008_II Y:/server/EFFIGY_PICS/Zdjecia2008_II
[(71,)]
Created folder 71
[(8,)]
this is root 8 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006
about to insert 8 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006 20101105_CUBA_ZDJECIA Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006/20101105_CUBA_ZDJECIA
[(72,)]
Created folder 72
about to insert 8 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006 CUBA 2012 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006/CUBA 2012
[(73,)]
Created folder 73
[(9,)]
this is root 9 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\20101105_CUBA_ZDJECIA
about to insert 9 Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\20101105_CUBA_ZDJECIA 100OLYMP Y:/server/EFFIGY_PICS\FILMY_MAMY_20121006\201

Created folder 118
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20070412Mama WiezaCNTower Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20070412Mama WiezaCNTower
[(119,)]
Created folder 119
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20070510_FarmaAdamMama Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20070510_FarmaAdamMama
[(120,)]
Created folder 120
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20070521_Cottage Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20070521_Cottage
[(121,)]
Created folder 121
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20070527_sylviaUrodziny Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20070527_sylviaUrodziny
[(122,)]
Created folder 122
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20070710_KidsPool Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20070710_KidsPool
[(123,)]
Created folder 123
about to insert 11 Y:/server/EFFIGY_PICS\Zdjecia\Minolta 20071010_Zosia_HighPark_Chomik Y:/server/EFFIGY_PICS\Zdjecia\Minolta/20071010_Zosia_HighPark_

[(20,)]
this is root 20 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\Barbican Drive Neighbourhoods BBQ Aug 5 2002
about to insert 20 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\Barbican Drive Neighbourhoods BBQ Aug 5 2002 20020701 Neghbourhoog Barbican Touch Ups Y:/server/EFFIGY_PICS\Zdjecia\Minolta\Barbican Drive Neighbourhoods BBQ Aug 5 2002/20020701 Neghbourhoog Barbican Touch Ups
[(161,)]
Created folder 161
about to insert 20 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\Barbican Drive Neighbourhoods BBQ Aug 5 2002 Originals Y:/server/EFFIGY_PICS\Zdjecia\Minolta\Barbican Drive Neighbourhoods BBQ Aug 5 2002/Originals
[(162,)]
Created folder 162
[(21,)]
this is root 21 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\My Jams
about to insert 21 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\My Jams 2003 Ela niania Y:/server/EFFIGY_PICS\Zdjecia\Minolta\My Jams/2003 Ela niania
[(163,)]
Created folder 163
about to insert 21 Y:/server/EFFIGY_PICS\Zdjecia\Minolta\My Jams Share Y:/server/EFFIGY_PICS\Zdjecia\Minolta\My Jams/Share
[

[(34,)]
this is root 34 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf
about to insert 34 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf Catalina Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf/Catalina
[(203,)]
Created folder 203
[(35,)]
this is root 35 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf\Catalina
about to insert 35 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf\Catalina localhost Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\conf\Catalina/localhost
[(204,)]
Created folder 204
[(36,)]
this is root 36 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server
about to insert 36 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server classes Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server/classes
[(205,)]
Created folder 205
about to insert 36 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server lib Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server/lib
[(206,)]
Created folder 206
about to insert 36 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\server webapps Y:/server/EF

[(241,)]
Created folder 241
about to insert 46 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples error Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples/error
[(242,)]
Created folder 242
about to insert 46 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples forward Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples/forward
[(243,)]
Created folder 243
about to insert 46 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples images Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples/images
[(244,)]
Created folder 244
about to insert 46 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples include Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\src\jakarta-servletapi-5\jsr152\examples/include
[(245,)]
Created folder 245
about to insert 46 Y:/server/EFFIGY_PI

...

[(270,)]
this is root 270 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\funcspecs
about to insert 270 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\funcspecs printer Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\catalina\funcspecs/printer
[(841,)]
Created folder 841
[(271,)]
this is root 271 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\config
about to insert 271 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\config printer Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\config/printer
[(842,)]
Created folder 842
[(272,)]
this is root 272 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jasper
about to insert 272 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jasper docs Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\jasper/docs
[(843,)]
Created folder 843
[(273,)]
this is root 273 Y:/server/EFFIGY_PICS\Zdje

[(286,)]
this is root 286 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi
about to insert 286 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi javax Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi/javax
[(868,)]
Created folder 868
about to insert 286 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi resources Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi/resources
[(869,)]
Created folder 869
[(287,)]
this is root 287 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax
about to insert 287 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax servlet Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax/servlet
[(870,)]
Created folder 870
[(288,)]
this is root 288 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\webapps\tomcat-docs\servletapi\javax\servlet
about to inser

Created folder 898
about to insert 299 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts util Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts/util
[(899,)]
Created folder 899
[(300,)]
this is root 300 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts\taglib
about to insert 300 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts\taglib bean Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts\taglib/bean
[(900,)]
Created folder 900
about to insert 300 Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts\taglib html Y:/server/EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA\work\Catalina\localhost\admin\loader\org\apache\struts\taglib/html
[(901,)]
Created folder 901
about to insert 300 Y:/server/EFFIGY_PICS\Zdje

Created folder 938
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20100905 WoodsCottage_house_Chrzest Y:/server/EFFIGY_PICS\Zdjecia2008_II/20100905 WoodsCottage_house_Chrzest
[(939,)]
Created folder 939
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20101101 4062 Barbican Drive Property Pics Y:/server/EFFIGY_PICS\Zdjecia2008_II/20101101 4062 Barbican Drive Property Pics
[(940,)]
Created folder 940
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20101101 HOUSE_4062_BARBICAN Y:/server/EFFIGY_PICS\Zdjecia2008_II/20101101 HOUSE_4062_BARBICAN
[(941,)]
Created folder 941
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20101215_museum_cousins Y:/server/EFFIGY_PICS\Zdjecia2008_II/20101215_museum_cousins
[(942,)]
Created folder 942
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20101226_NartySylwester Y:/server/EFFIGY_PICS\Zdjecia2008_II/20101226_NartySylwester
[(943,)]
Created folder 943
about to insert 305 Y:/server/EFFIGY_PICS\Zdjecia2008_II 20

[(978,)]
Created folder 978
[(311,)]
this is root 311 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK
about to insert 311 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK big Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK/big
[(979,)]
Created folder 979
about to insert 311 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK Gotowe Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK/Gotowe
[(980,)]
Created folder 980
about to insert 311 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK Pozegnanie Lata 2008 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forNK/Pozegnanie Lata 2008
[(981,)]
Created folder 981
about to insert 311 Y:/server/EFFIGY_PICS\Zdjecia2008_II\1_DUPLICATES MAYBE_ZDJECIA_part_polska\Zdjecia\forN

In [ ]:
id = getID("loc_root","root_id","root","C:/Users/user/Pictures/2018-06")
id

In [ ]:
id = getID("loc_folder","folder_id","folder_path","C:/Users/user/Pictures/2018-06")
id


In [ ]:
getSampleData()

In [ ]:
path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/test.jpg"
from PIL import Image
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 

# prints format of image 
print(img.format) 

# prints mode of image 
print(img.mode) 

In [ ]:
for r, d, f in os.walk(path):
    print ("directory is: ", d)
    

In [ ]:
for f in files:
    print(f)
    

In [ ]:
for r, d, f in os.walk(path):
    print(f)

In [ ]:
from PIL import Image
import glob, os
size = 128, 128
for infile in glob.glob(path):
    file, ext = os.path.splitext(infile)
    im = Image.open(infile)
    im.thumbnail(size)
    im.save(file + ".thumbnail1.png", "PNG")

In [ ]:

path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/geotag.jpg"

# Python program to read 
# image using PIL module 
  
# importing PIL 
from PIL import Image 
  
# Read image 
img = Image.open(path) 
  
# Output Images 
#img.show() 
  
# prints format of image 
print(img.format) 
  
    
    

# prints mode of image 
print(img.mode) 
dic = img._getexif()
width = dic.get(40963)
height = dic.get(40962)
camera = dic.get(271)
model = dic.get(272)
dttaken = dic.get(36868)
dtmod= dic.get(36867)
fstop= dic.get(42036)
exposure = dic.get(40960) 
orientation=dic.get(274)
flash = dic.get(7385)
shutter = dic.get(37377)
apperture=dic.get(37378) 
isospeed=dic.get(34855) 
digzoom=dic.get(41988)
exif=""


In [ ]:
dttaken

In [ ]:
exif=""
for x in dic:
     if x != 37500:
        exif= exif + (str(x) +  ":" +  str(dic.get(x)) + chr(13) +chr(10) )  
exif

In [ ]:
for x in dic:
    if x != 37500:
     print(x,":", dic.get(x))

In [ ]:
import os
path = 'C:/Users/user/Pictures'
roots = []
# r=root, d=directories, f = files
for r,d,f in os.walk(path):
    print (r,d)
    for f in d:
       print(f)
   